In [ ]:
# # This cell runs bowtie locally, uncomment last line to run the command and test your bowtie
# 
# import os
# # wget index and SRR043378 directly into project folder
# threads = 4  # Can be changed
# # folder name then file name
# bowtieIndex = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
#               "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"
# # fastq read file
# reads = "SRR043378_1.fastq/data"
# cmd = "bowtie2 -p "+str(threads)+" -x " + bowtieIndex + " -U "+reads+" -S out.sam"
# 
# 
# os.system(cmd)

In [ ]:
# # This cell tests pyspark
# 
# import pyspark
# import random
# sc = pyspark.SparkContext(appName="Pi")
# num_samples = 100000000
# 
# 
# def inside(p):     
#     x, y = random.random(), random.random()
#     return x*x + y*y < 1
# 
# 
# count = sc.parallelize(range(0, num_samples)).filter(inside).count()  
# pi = 4 * count / num_samples
# print(pi)
# sc.stop()

3.1413872


In [1]:
# import pyspark
# import os
# 
# # new sparkContext
# sc = pyspark.SparkContext(appName="DNA")

In [2]:
#######################################################################################################
# TODO: Put files in hdfs, split file into parts for each worker to process (should be able to just run
# bowtie2 command on each mapper node). Then run mapper, should output however many files there are 
# nodes, then combine the output files. 
#######################################################################################################

 
# # Basis for map function, output files probably need different names for different nodes and file 
# # locations need to be changed to the hdfs location
# threads = 1  # Can be changed
# # folder name then file name
# bowtieIndex = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
#               "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"
# # fastq read file
# reads = "SRR043378_1.fastq/data"
# cmd = "bowtie2 -p " + str(threads) + " -x " + bowtieIndex + " -U " + reads + " -S out.sam"


In [1]:
#import findspark
#findspark.init('/home/taylor/spark')

from pyspark import SparkConf, SparkContext
import math
from pyspark.sql import SQLContext


conf = SparkConf().setMaster("local").setAppName("DNA")

sc = SparkContext(conf=conf)

In [2]:
# creates tuples with format (line number, string)
raw_input = (sc.textFile("SRR043378_1.fastq/data")).zipWithIndex()

In [ ]:
# create tuple with format (read number, string)
indexed_raw_input = raw_input.map(lambda x: (math.floor(x[1]/4), x[0]))

In [4]:
# Function to pass to map function that concatenates the 4 strings
def make_reads(iterable_read):
    x = 0
    line1 = ""
    line2 = ""
    line3 = ""
    line4 = ""
    for line in iterable_read:
        if x == 0:
            line1 = line
        if x == 1:
            line2 = line
        if x == 2:
            line3 = line
        if x == 3:
            line4 = line
        x = x+1
    return "%s\n%s\n%s\n%s\n" % (line1, line2, line3, line4)

In [6]:
reads = indexed_raw_input.groupByKey().mapValues(lambda x: make_reads(x))

In [8]:
test = reads.take(10)

@SRR043378.1 618UWAAXX100205:6:1:995:4021/1
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
+
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [13]:
# Test get one record
print(test[0][1])

@SRR043378.1 618UWAAXX100205:6:1:995:4021/1
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
+
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [ ]:
sc.stop()